In [18]:
import re
import os
import string

from gensim.models import Word2Vec
from gensim.similarities.index import AnnoyIndexer

### Load corpus from the text file

In [19]:
with open('sinhala.txt', 'r') as fp:
    text = fp.read()
    print(text[:200])

අමාත්‍යාංශ කටයුතු මෙහෙයවීමට වාසුදේව නානායක්කාර මැතිතුමා ගරු ජාතික භාෂා හා සමාජ ඒකාබද්ධතා අමාත්‍යවරයා ලෙසද , පලනි තිගම්බරම් මැතිතුමා ගරු ජාතික භාෂා හා සමාජ ඒකාබද්ධතා නියෝජ්‍ය අමාත්‍යවරයා ලෙස ද එම් . එස


### Clean data

In [21]:
filtered_text = ''.join(
            [i for i in text if (not i.isdigit() and i not in set(string.punctuation) and i not in ['“', '”'])])
re.sub(' +', ' ', filtered_text)
sentences = filtered_text.split('\n')
word_list = [line.split(' ') for line in sentences]

def remove_null(line):
    while '' in line:
        line.remove('')
    line = [ word for word in line if not word.isalpha()]
    return line
    
word_list = [ remove_null(line) for line in word_list]
word_list[:2]

[['අමාත්\u200dයාංශ',
  'කටයුතු',
  'මෙහෙයවීමට',
  'වාසුදේව',
  'නානායක්කාර',
  'මැතිතුමා',
  'ගරු',
  'ජාතික',
  'භාෂා',
  'හා',
  'සමාජ',
  'ඒකාබද්ධතා',
  'අමාත්\u200dයවරයා',
  'ලෙසද',
  'පලනි',
  'තිගම්බරම්',
  'මැතිතුමා',
  'ගරු',
  'ජාතික',
  'භාෂා',
  'හා',
  'සමාජ',
  'ඒකාබද්ධතා',
  'නියෝජ්\u200dය',
  'අමාත්\u200dයවරයා',
  'ලෙස',
  'එම්',
  'එස්',
  'වික්\u200dරමසිංහ',
  'මිය',
  'අමාත්\u200dයාංශ',
  'ලේකම්',
  'ලෙස',
  'පත්',
  'ලදි'],
 ['මෙම',
  'අමාත්\u200dයාංශය',
  'අංක',
  'බුත්ගමුවපාර',
  'රාජගිරිය',
  'ස්ථානයේ',
  'පිහිටා']]

### Train CBOW and skip-gram models

In [29]:
 def train_word_to_vec_v2(skipgram=0):
        # train model
        
        if skipgram == 0:
            model_name = 'skipgram_model.bin'
        else:
            model_name = 'cbow_model.bin'
        if os.path.exists(model_name):
            print('load existing model - {}'.format(model_name))
            new_model = Word2Vec.load(model_name)
        else:
            model = Word2Vec(word_list, size=100, window=5, min_count=1, workers=4, sg=skipgram)
            words = list(model.wv.vocab)
            model.save(model_name)
            # load model
            new_model = Word2Vec.load(model_name)
            # print(new_model)
        return new_model

In [30]:
model_cbow = train_word_to_vec_v2(1)
model_skipgram = train_word_to_vec_v2(0)

### Checking word similarity

In [31]:
vector1 = model_cbow.wv["කටයුතු"]
vector2 = model_cbow.wv["සාකච්ඡා"]
print(model_cbow.wv.distance('කටයුතු', 'ඉදිරියේදී'))
print(model_cbow.wv.most_similar([vector1]))

0.3064647316932678
[('කටයුතු', 0.9999999403953552), ('සැලසුම්', 0.7395913600921631), ('අලුත්වැඩියා', 0.7228710651397705), ('කඩිනමින්', 0.7158126831054688), ('සිදු', 0.7099413275718689), ('සංවිධානය', 0.7082185745239258), ('විධිමත්', 0.7065615653991699), ('සාකච්ඡා', 0.7022934556007385), ('දෝෂ', 0.701819658279419), ('සම්පාදනය', 0.6998875737190247)]


### Get most frequent words which is longer than 4 charactors


In [41]:
common_words = [ word for word in model_cbow.wv.index2entity[:100] if len(word) > 3 ][:10]
print(common_words)

['සඳහා', 'යුතු', 'කිරීම', 'කටයුතු', 'විසින්', 'සඳහන්', 'ලේකම්', 'වැනි', 'කවරේද', 'යටතේ']


In [48]:
model_name = ['CBOW', 'skip-gram']
for word in common_words:
    print("----------------------------------------------------\nWord :  {}".format(word))
    for i, model in enumerate([model_cbow, model_skipgram]):
        vector = model.wv[word]
        sim_words = model.wv.most_similar([vector])
        print("{:10} -> {}".format(model_name[i], ', '.join([ ele[0] for ele in sim_words])))

----------------------------------------------------
Word :  සඳහා
CBOW       -> සඳහා, සදහා, වලින්, වෙනුවෙන්, පිණිස, යාවත්කාලීන, කිරීමටත්, අවශ්‍යතා, පදනම්, කරගෙන
skip-gram  -> සඳහා, කිරීම, යොදා, පියවා, ආරම්භ, සැපයුම, බඳවා, ක්‍රියාමාර්ගයක්, අපේක්ෂිත, ගෙන්වා
----------------------------------------------------
Word :  යුතු
CBOW       -> යුතු, යුතුය, හැකි, යුත්තේ, කෙරෙන, අදාළ, දැක්වෙන, කල්, නොහැකි, ලියා
skip-gram  -> යුතු, ඉල්ලුම්පත්‍රයේ, ඉල්ලුම්පත්‍රය, ලිපියකින්, දිය, නොගෙවිය, යුතුය, ගම්‍ය, විය, සැලකිය
----------------------------------------------------
Word :  කිරීම
CBOW       -> කිරීම, කිරීමට, කිරීම්, කිරීමේ, කිරීමේදී, කිරීමටත්, කෙරෙන, කොට, සැලැස්ම, ප්‍රකාශයට
skip-gram  -> කිරීම, සඳහා, ගිවිසුම, කටයුත්තක්, කි‍්‍රයාත්මක, සකස්, ආරක්ෂා, වැඩපිළිවෙලක්, බලපත‍්‍රයක්, සිදු
----------------------------------------------------
Word :  කටයුතු
CBOW       -> කටයුතු, සැලසුම්, අලුත්වැඩියා, කඩිනමින්, සිදු, සංවිධානය, විධිමත්, සාකච්ඡා, දෝෂ, සම්පාදනය
skip-gram  -> කටයුතු, මගපෙන්වීම, අවශ්‍ය, දීමට, කරන්නේද,

In [16]:
from gensim.test.utils import datapath
from gensim.models.fasttext import load_facebook_model

cap_path = datapath("/home/dulanj/MEGAsync/MSc/msc-code/Sem2/NLP/fastText/cc.si.300.bin.gz")
fb_model = load_facebook_model(cap_path)

In [52]:
for word in common_words:
    print("------------------------------------------------------------------------------\nWord : {}".format(word))
    vector = fb_model.wv[word]
    sim_words = fb_model.wv.most_similar([vector])
    print( ', '.join([ ele[0] for ele in sim_words]))
        
# vector1 = fb_model.wv["කටයුතු"]
# fb_model.wv.most_similar([vector1])

------------------------------------------------------------------------------
Word : සඳහා
සඳහා, සදහා, සඳහා‍, කිරීම, සඳහාඳ, සඳහාවු, සඳහාඑජ, සඳහා1.54ක්, සඳහාමද, සඳහාඑම
------------------------------------------------------------------------------
Word : යුතු
යුතු, හැකි, යුතුය, යුතුමුත්, යුතුම, යුතුමව, යුතුයි.අවශ්‍ය, යුතුමද, විය, නොහැකි
------------------------------------------------------------------------------
Word : කිරීම
කිරීම, කිරීමට, කිරීම.ඒ, කිරීම‍‍, ගැනීම, කිරීමල, කිරීමේ, කිරීම3, වීම, කිරීමි
------------------------------------------------------------------------------
Word : කටයුතු
කටයුතු, කටයුතුව, කටයුතුවලදීත්, කලාකටයුතු, කටයුතුවලද, කටයුතුවලදීද, කටයුතුකල, කටයුතුකලා, සේවාකටයුතු, සමගකටයුතු
------------------------------------------------------------------------------
Word : විසින්
විසින්, මගින්, විසින්ම, වෙත, විසින, විසිනි, මහතාවිසින්, මඟින්, හර්ෂල්විසින්, තමාවිසින්ම
------------------------------------------------------------------------------
Word : සඳහන්
සඳහන්, සඳහන්ඉඟි, සදහ